쿠팡에서 데이터 긁어오기

1. 패키지 설치

In [ ]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

2. 데이터 수집할 웹 페이지

In [ ]:
# 수집할 컨텐츠가 있는 웹 페이지 주소
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"

# 브라우저 버전정보
userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

# 접속 객체 생성
# request는 직접 header 정보 갱신이 불가능해 session 생성
session = requests.Session()

# 접속 객체에 부가정보(header) 삽입
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})

# 생성한 접속 객체를 활용해 API에 접속
r = session.get(url)

# 접속 실패 시 에러 코드와 에러 메시지 출력
if r.status_code != 200:
    msg = "[%d Error] %s 에러 발생"%(r.status_code, r.reason)
    raise Exception(msg)

# 인코딩 형식을 지정해 beautifulsoup 객체를 생성
r.encoding = 'utf-8'
# r 변수에 저장된 text 정보를 시각화, 확인
# print(r.text)

# BeautifulSoup 객체 soup를 생성
soup = BeautifulSoup(r.text)
soup

3. 필요한 내용 추출\
dataList에서 soup에 담겨지는 select의 범위를 비교해 볼 것

In [ ]:
# class가 post인 요소의 데이터 추출
dataList = soup.select(".search-content.search-content-with-feedback")
# dataList = soup.select('.search-product-wrap')
dataList

아래는 내가 짠 코드

In [ ]:
# 수집한 결과가 저장될 빈 리스트 생성
result = []

for data in dataList:
    # print(data)
    # 상품 이름 추출
    titleEl = data.select(".name")
    # print(titleEl)
    title = titleEl[0].text.strip()
    # print(title)
    # for index1, title in enumerate(titleEl):
    #     print(index1, title.text.strip())
    

    # 가격 추출
    priceEl = data.select(".base-price")
    # print(priceEl)
    price = priceEl[0].text.strip()
    # print(price)


    # 할인율 추출
    discountEl = data.select(".instant-discount-rate")
    # print(discountEl)
    discount = discountEl[0].text.strip()
    # print(discount)


    # 별점 추출
    starEl = data.select(".star")
    # print(starEl)
    star = starEl[0].text.strip()
    # print(star)


    # # 이미지 추출
    # imgEl = data.select(".search-product-wrap-img")
    # print(imgEl)
    # img = imgEl[0].text.strip()
    # # print(img)

    # 추출한 내용을 딕셔너리로 병합
    item = {"이름": title, "가격": price, "할인율": discount, "별점":star}

    # 리스트에 추출한 내용 삽입
    result.append(item)
result

# 현재 위치의 코드는 dataList = soup.select(".search-product-wrap")를 사용할 경우
# IndexError: list index out of range에러가 발생
# 특정 값이 공백인 경우의 처리를 해줘야 하는 것으로 생각함

In [ ]:
# 수집한 정보를 excel 형식으로 저장
df = DataFrame(result)
df.to_excel("쿠팡상품정보.xlsx", index=False)
df